## What this file does
This file is an edited version of sample.ipynb. It splits the training data into a training and validation set, and shows how to compute the RMSE of a model on the validation data. (Note that this split happens towards the end of the file, after you have created new features, if you choose to do so.)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from __future__ import print_function
from rdkit import Chem 
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
import pickle

In [2]:
"""
Read in train/validate and test as Pandas DataFrames
"""
df_train = pd.read_csv("train")
df_test = pd.read_csv("test")

In [3]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.98


In [4]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [5]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [6]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


## Feature engineering

In [7]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [11]:
arrays_for_df = []

In [35]:
%%time
#the columns that will appear in the final dataframe
##df_touse_columns = df_all.columns[1:].tolist()+['double','num_unique_elem','num_valence_elec',"ring_count",'H_donors','H_acceptors','num_atoms','len']
#split the dataframe into 20 dataframes for creating the molecule column
arraylist = [] #array list appended within for loop
split_df_all = np.array_split(df_all[1600000:],4) #increase by 200000 each time
for df in split_df_all: 
    df['molecule'] = map(Chem.MolFromSmiles, df['smiles'].values)
    #number double bonds
    df['double'] = map(lambda m : sum(np.array([b.GetBondType() for b in m.GetBonds()])==2), df['molecule'])
    #number unique elements
    df['num_unique_elem'] = map(lambda m : len(set([atom.GetAtomicNum() for atom in m.GetAtoms()])), df['molecule'])
    #number valence electrons 
    df['num_valence_elec'] = map(lambda m : Descriptors.NumValenceElectrons(m), df['molecule'])
    #number rings
    df['ring_count'] = map(lambda m : Descriptors.RingCount(m), df['molecule'])
    #number H donors
    df['H_donors'] = map(lambda m : Descriptors.NumHDonors(m), df['molecule'])
    #number H acceptors
    df['H_acceptors'] = map(lambda m : Descriptors.NumHAcceptors(m), df['molecule'])
    #number atoms
    df['num_atoms'] = map(lambda m : len(m.GetAtoms()), df['molecule'])
    #length of smiles string
    df['len'] = map(len, df['smiles'].values)
    array = df.drop(['molecule','smiles'],axis=1).values
    arraylist.append(array)
array_concat = np.concatenate(arraylist,axis=0)
arrays_for_df.append(array_concat)

CPU times: user 7min 15s, sys: 38.2 s, total: 7min 53s
Wall time: 9min 52s


In [36]:
len(arrays_for_df)

7

In [37]:
%%time
#pickling
pickle.dump(arrays_for_df[6],open("arrays_for_df_6.p","wb"))
##array_touse = np.concatenate(arrays_for_df)
##pickle.dump(array_touse, open("array_touse.p", "wb"))
##pickle.dump(df_touse_columns, open("column_names.p","wb"))
#note: arrays_for_df_4.p is really 5 (1300000-1600000); 4 would be 1000000-1300000. 

CPU times: user 1min 6s, sys: 9.16 s, total: 1min 15s
Wall time: 2min 17s


In [38]:
%%time
array_touse = np.concatenate(arrays_for_df,axis=0)

CPU times: user 1.78 s, sys: 4.27 s, total: 6.06 s
Wall time: 15 s


In [41]:
%%time
np.savetxt("array_touse.csv", array_touse, delimiter=",")

CPU times: user 3min 45s, sys: 28.4 s, total: 4min 14s
Wall time: 6min 42s


In [44]:
%%time
##pickle.dump(array_touse, open("array_touse.p", "wb"))
pickle.dump(df_touse_columns, open("column_names.p","wb"))

CPU times: user 2.63 ms, sys: 2.12 ms, total: 4.75 ms
Wall time: 16.6 ms


In [ ]:
'''
atomic_numbers = [7,8,14,16,34]
#DO FEATURE CREATION
dflist = []
split_df_all = np.array_split(df_all,20)
for df in split_df_all: 
    #number double bonds
    df['double'] = map(lambda m : sum(np.array([b.GetBondType() for b in m.GetBonds()])==2), df['molecule'])
    #number unique elements
    df['num_unique_elem'] = map(lambda m : len(set([atom.GetAtomicNum() for atom in m.GetAtoms()])), df['molecule'])
    #number valence electrons 
    df_temp['num_valence_elec'] = map(lambda m : Descriptors.NumValenceElectrons(m), df_temp['molecule'])
    #number rings
    df_temp['ring_count'] = map(lambda m : Descriptors.RingCount(m), df_temp['molecule'])
    dflist.append(df)
    #number H donors
    df_temp['H_donors'] = map(lambda m : Descriptors.NumHDonors(m), df_temp['molecule'])
    #number H acceptors
    df_temp['H_acceptros'] = map(lambda m : Descriptors.NumHAcceptors(m), df_temp['molecule'])
    #whether each element appears in the molecule 
    for num in atomic_numbers: 
        df_temp["atmnum_"+str(num)] = np.zeros(len(df_temp))
    #for each molecule, set the value of each atom
    i=0
    for m in df_temp['molecule']:
        ##m = row['molecule']
        elements = set([atom.GetAtomicNum() for atom in m.GetAtoms()])
        for element in elements:
            df_temp.set_value(i,"atmnum_"+str(element),1)
        i+=1

df_all = pd.concat(dflist)

In [ ]:
import pickle 
with open('filename.pickle', 'wb') as handle:
    pickle.dump(df_all, handle)

In [ ]:
'''
%%time
#create a column with the rdkit molecule object 
df_all['molecule'] = map(Chem.MolFromSmiles, df_all['smiles'].values)

In [ ]:
#Drop the 'smiles' and 'molecule' columns
##df_all = df_all.drop(['smiles','molecule'], axis=1)
X_train_df = df_touse[:test_idx]
X_test_df = df_touse[test_idx:]
##print "Train features: " , X_train.shape
##print "Train gap: " , Y_train.shape
##print "Test features: " , X_test.shape

## Training-validation split
Having done feature engineering, we split the training data into a training (30%) and validation (70%) set:

In [ ]:
%%time
val_train_split = np.random.choice([0, 1], size=(len(X_train_df)), p=[.3, .7])
X_train_df["training"] = val_train_split
X_validate_df =  X_train_df[X_train_df['training']==0].drop(['training'],axis=1) #create a new X_validate dataframe
X_train_df =  X_train_df[X_train_df['training']==1].drop(['training'],axis=1) #redefine the X_train dataframe so that it is just training data
#create np matrices of data
X_validate = X_validate_df.values 
X_train = X_train_df.values
X_test = X_test_df.values
#split the Y data
Y_validate = [Y_train[i] for i in range(len(Y_train)) if val_train_split[i]==0] 
Y_train = [Y_train[i] for i in range(len(Y_train)) if val_train_split[i]==1]


In [ ]:
## The first value of X_train.shape should equal the length of Y_train
print(X_train.shape)
print(len(Y_train))

##Computing RMSE
The sklearn function below computes RMSE, which is the loss function we are minimizing in this practical.

In [ ]:
%%time 
LR = LinearRegression()
LR.fit(X_train, Y_train)
#check RMSE on validation set 
LR_val_pred = LR.predict(X_validate)
LR_MSE = mean_squared_error(Y_validate, LR_val_pred)

LR_pred = LR.predict(X_test)
print(LR_MSE)

In [ ]:
%%time 
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_val_pred = RF.predict(X_validate)
RF_MSE = mean_squared_error
RF_pred = RF.predict(X_test)


In [ ]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [ ]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)